In [1]:
!pip install tfx


     |████████████████████████████████| 2.4 MB 5.1 MB/s 
     |████████████████████████████████| 1.4 MB 39.3 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 135 kB 56.3 MB/s 
     |████████████████████████████████| 19.0 MB 21 kB/s 
     |████████████████████████████████| 6.5 MB 45.5 MB/s 
     |████████████████████████████████| 40 kB 5.6 MB/s 
     |████████████████████████████████| 1.3 MB 60.8 MB/s 
     |████████████████████████████████| 454.4 MB 9.5 kB/s 
     |████████████████████████████████| 147 kB 47.5 MB/s 
     |████████████████████████████████| 17.7 MB 76 kB/s 
     |████████████████████████████████| 1.2 MB 48.3 MB/s 
     |████████████████████████████████| 1.7 MB 37.8 MB/s 
     |████████████████████████████████| 189 kB 51.0 MB/s 
     |████████████████████████████████| 1.7 MB 55.5 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 
     |████████████████████████████████| 406 kB 64.7 MB/s 
     |████████████████

# Import libraries and packages

In [1]:
import tensorflow as tf 
import tensorflow_data_validation as tfdv 

from tfx.components import CsvExampleGen, ExampleValidator, SchemaGen, StatisticsGen, Transform
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from google.protobuf.json_format import MessageToDict 

import os 
import pprint
pp = pprint.PrettyPrinter()

In [3]:
!pip install fsspec

     |████████████████████████████████| 123 kB 4.9 MB/s 


# Preview the dataset

In [4]:
# preview the first rows of the data
import pandas as pd 
import dask.dataframe as dd
df = dd.read_csv('/content/drive/MyDrive/MLOPS/tft/data/seoul/*', encoding='latin-1')
df.head()

,Date,Rented Bike Count,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Dew point temperature(°C),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Seasons,Holiday,Functioning Day
0,01/12/2017,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
1,01/12/2017,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
2,01/12/2017,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,Winter,No Holiday,Yes
3,01/12/2017,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,Winter,No Holiday,Yes
4,01/12/2017,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,Winter,No Holiday,Yes


In [31]:
cols = [col.lower().split('(')[0].rstrip().replace(' ', '_') for col in df.columns]
cols

['date',
 'rented_bike_count',
 'hour',
 'temperature',
 'humidity',
 'wind_speed',
 'visibility',
 'dew_point_temperature',
 'solar_radiation',
 'rainfall',
 'snowfall',
 'seasons',
 'holiday',
 'functioning_day']

In [33]:
df.to_csv('/content/drive/MyDrive/MLOPS/tft/data/seoul/transformed/seoul.csv', index=False, header=cols)

['/content/drive/MyDrive/MLOPS/tft/data/seoul/transformed/seoul.csv/0.part']

# Define Paths

In [34]:
# location of the pipeline metadata store
_pipeline_root = '/content/pipeline'

# directory of raw data files
_data_root = '/content/drive/MyDrive/MLOPS/tft/data/seoul/transformed/seoul.csv/'


# Create the Interactive Context

In [35]:
# Initialize the InteractiveContext with a local sqlite file.
# If you leave `_pipeline_root` blank, then the db will be created in a temporary directory.
# You can safely ignore the warning about the missing config file.
context = InteractiveContext(pipeline_root=_pipeline_root)

# Ingest data - CsvExampleGen

In [36]:
# Instantiate ExampleGen with the input csv dataset
example_gen = CsvExampleGen(input_base=_data_root)

In [37]:
context.show(example_gen)

CsvExampleGen(spec: <tfx.types.standard_component_specs.FileBasedExampleGenSpec object at 0x7f09354e71d0>, executor_spec: <tfx.dsl.components.base.executor_spec.BeamExecutorSpec object at 0x7f09354e7750>, driver_class: <class 'tfx.components.example_gen.driver.FileBasedDriver'>, component_id: CsvExampleGen, inputs: {}, outputs: {'examples': Channel(
    type_name: Examples
    artifacts: []
    additional_properties: {}
    additional_custom_properties: {}
)})

In [38]:
# execute the component
context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 4
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 4
        type_id: 14
        uri: "/content/pipeline/CsvExampleGen/examples/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "file_format"
          value {
            string_value: "tfrecords_gzip"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:642933,xor_checksum:1632956555,sum_checksum:1632956555"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [39]:
# get the artifact object
artifact = example_gen.outputs['examples'].get()[0]

# print split names and uri
print(f'split names: {artifact.split_names}')
print(f'artifact uri: {artifact.uri}')

split names: ["train", "eval"]
artifact uri: /content/pipeline/CsvExampleGen/examples/4


In [40]:
# Get the URI of the output artifact representing the training examples
train_uri = os.path.join(artifact.uri, 'Split-train')

# See the contents of the `train` folder
!ls {train_uri}

data_tfrecord-00000-of-00001.gz


In [41]:
# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [42]:
# Define a helper function to get individual examples
def get_records(dataset, num_records):
    '''Extracts records from the given dataset.
    Args:
        dataset (TFRecordDataset): dataset saved by ExampleGen
        num_records (int): number of records to preview
    '''
    
    # initialize an empty list
    records = []
    
    # Use the `take()` method to specify how many records to get
    for tfrecord in dataset.take(num_records):
        
        # Get the numpy property of the tensor
        serialized_example = tfrecord.numpy()
        
        # Initialize a `tf.train.Example()` to read the serialized data
        example = tf.train.Example()
        
        # Read the example data (output is a protocol buffer message)
        example.ParseFromString(serialized_example)
        
        # convert the protocol bufffer message to a Python dictionary
        example_dict = (MessageToDict(example))
        
        # append to the records list
        records.append(example_dict)
        
    return records

In [43]:
# Get 3 records from the dataset
sample_records = get_records(dataset, 3)

# Print the output
pp.pprint(sample_records)

[{'features': {'feature': {'date': {'bytesList': {'value': ['MDEvMTIvMjAxNw==']}},
                           'dew_point_temperature': {'floatList': {'value': [-17.6]}},
                           'functioning_day': {'bytesList': {'value': ['WWVz']}},
                           'holiday': {'bytesList': {'value': ['Tm8gSG9saWRheQ==']}},
                           'hour': {'int64List': {'value': ['0']}},
                           'humidity': {'int64List': {'value': ['37']}},
                           'rainfall': {'floatList': {'value': [0.0]}},
                           'rented_bike_count': {'int64List': {'value': ['254']}},
                           'seasons': {'bytesList': {'value': ['V2ludGVy']}},
                           'snowfall': {'floatList': {'value': [0.0]}},
                           'solar_radiation': {'floatList': {'value': [0.0]}},
                           'temperature': {'floatList': {'value': [-5.2]}},
                           'visibility': {'int64List': {'valu

# StatisticsGen

In [44]:
# Instantiate StatisticsGen with the Examplegen ingested dataset
statistics_gen = StatisticsGen(
    examples=example_gen.outputs['examples']
)

# Execute the comment
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 5
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 5
        type_id: 16
        uri: "/content/pipeline/StatisticsGen/statistics/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

# SchemaGen

In [45]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics']
)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 6
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 6
        type_id: 18
        uri: "/content/pipeline/SchemaGen/schema/6"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [46]:
# visualise the schema
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'date',BYTES,required,,-
'functioning_day',STRING,required,,'functioning_day'
'holiday',STRING,required,,'holiday'
'seasons',STRING,required,,'seasons'
'dew_point_temperature',FLOAT,required,,-
'hour',INT,required,,-
'humidity',INT,required,,-
'rainfall',FLOAT,required,,-
'rented_bike_count',INT,required,,-


/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'functioning_day',"'No', 'Yes'"
'holiday',"'Holiday', 'No Holiday'"
'seasons',"'Autumn', 'Spring', 'Summer', 'Winter'"


# Visualising the statistics

In [48]:
train_stats = tfdv.load_stats_binary(statistics_gen.outputs['statistics']._artifacts[0].uri + '/Split-train/FeatureStats.pb')
eval_stats = tfdv.load_stats_binary(statistics_gen.outputs['statistics']._artifacts[0].uri + '/Split-eval/FeatureStats.pb')
tfdv.visualize_statistics(lhs_statistics=eval_stats, rhs_statistics=train_stats,
                          lhs_name='EVAL_DATASET', rhs_name='TRAIN_DATASET')

Let's now move to the next step in the pipeline and see if there are any anomalies in the data.

# ExampleValidator

In [21]:
# Instantiate ExampleValidator with the StatisticsGen and SchemaGen ingested data
example_validator = ExampleValidator(
    statistics= statistics_gen.outputs['statistics'],
    schema= schema_gen.outputs['schema']
)

-  No anomalies detected
- Need to convert dates to hours

# Transform 

The [Transform](https://www.tensorflow.org/tfx/guide/transform) component performs feature engineering for both training and serving datasets. It uses the [TensorFlow Transform](https://www.tensorflow.org/tfx/transform/get_started) library introduced in the first ungraded lab of this week.

`Transform` will take as input the data from `ExampleGen`, the schema from `SchemaGen`, as well as a module containing the preprocessing function.

In this section, you will work on an example of a user-defined Transform code. The pipeline needs to load this as a module so you need to use the magic command `%% writefile` to save the file to disk. Let's first define a few constants that group the data's attributes according to the transforms we will perform later. This file will also be saved locally.

In [56]:
# set the constants module filename
import os
_seoul_constants_module_file = '/content/module_file/seoul_constants.py'

if not os.path.exists('/content/module_file'):
  os.makedirs('/content/module_file')

In [57]:
%%writefile {_seoul_constants_module_file}

# Features with string data types that will be converted to indices
CATEGORICAL_FEATURE_KEYS = ['functioning_day', 'holiday', 'seasons']

# Numerical features that are marked as continuous
NUMERIC_FEATURE_KEYS = ['hour','temperature','humidity','wind_speed',
                        'visibility','dew_point_temperature','solar_radiation','rainfall','snowfall']

# Feature that the model will predict
LABEL_KEY = 'rented_bike_count'

# Utility function for renaming the feature
def transformed_name(key):
    return key + '_xf'



Writing /content/module_file/seoul_constants.py


Next, you will work on the module that contains `preprocessing_fn()`. As you've seen in the previous lab, this function defines how you will transform the raw data into features that your model can train on (i.e. the next step in the pipeline). You will use the [tft module functions](https://www.tensorflow.org/tfx/transform/api_docs/python/tft) to make these transformations. 

In [72]:
_seoul_transform_module_file = '/content/module_file/seoul_transform.py'

In [73]:
%%writefile {_seoul_transform_module_file}

import tensorflow as tf  
import tensorflow_transform as tft 

import sys
sys.path.insert(0, '/content/module_file/' )
import seoul_constants

_NUMERIC_FEATURE_KEYS = seoul_constants.NUMERIC_FEATURE_KEYS
_CATEGORICAL_FEATURE_KEYS = seoul_constants.CATEGORICAL_FEATURE_KEYS
_LABEL_KEY = seoul_constants.LABEL_KEY
_transformed_name = seoul_constants.transformed_name


# Define the transformations
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.
    Args:
        inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
        Map from string feature key to transformed feature operations.
    """
  outputs = {}

  # Scale the features to the range of [0,1]
  for key in _NUMERIC_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.scale_to_0_1(inputs[key])
    
    

  
  # Convert strings to indices in a vocabulary
  for key in _CATEGORICAL_FEATURE_KEYS:
    outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(inputs[key])

  # Convert the label strings to an index
  outputs[_transformed_name(_LABEL_KEY)] = inputs[_LABEL_KEY]

  return outputs

Overwriting /content/module_file/seoul_transform.py


In [74]:
# Ignore TF warning messages
tf.get_logger().setLevel('ERROR')

# Instantiate the Transform component
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_seoul_transform_module_file))

# Run the component
context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 10
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 7
        type_id: 20
        uri: "/content/pipeline/Transform/transform_graph/7"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1632961734050
        last_update_time_since_epoch: 1632961779351
        , artifact_type: id: 20
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 8
        type_id: 14
        uri: "/content/pipeline/Transform/transformed_examples/7"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1632961734051
        last_update_time_since_epoch: 1632961779351
        , artifact_type: id: 14
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 9
        type_id: 21
        uri: "/content/pipeline/Transform/updated_analyzer_cache/7"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.2.0"
          }
        }
        state: LIVE
        create_time_since_epoch: 1632961734051
        last_update_time_since_epoch: 1632961779351
        , artifact_type: id: 21
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 10
        type_id: 18
        uri: "/content/pipeline/Transform/pre_transform_schema/7"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value

In [75]:
# Get the uri of the transform graph
transform_graph_uri = transform.outputs['transform_graph'].get()[0].uri

# List the subdirectories under the uri
os.listdir(transform_graph_uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [76]:
# Get the URI of the output artifact representing the transformed examples
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name) for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
transformed_dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

In [77]:
# Get 3 records from the dataset
sample_records_xf = get_records(transformed_dataset, 3)

# Print the output
pp.pprint(sample_records_xf)

[{'features': {'feature': {'dew_point_temperature_xf': {'floatList': {'value': [0.2155477]}},
                           'functioning_day_xf': {'int64List': {'value': ['0']}},
                           'holiday_xf': {'int64List': {'value': ['0']}},
                           'hour_xf': {'floatList': {'value': [0.0]}},
                           'humidity_xf': {'floatList': {'value': [0.37755102]}},
                           'rainfall_xf': {'floatList': {'value': [0.0]}},
                           'rented_bike_count_xf': {'int64List': {'value': ['254']}},
                           'seasons_xf': {'int64List': {'value': ['3']}},
                           'snowfall_xf': {'floatList': {'value': [0.0]}},
                           'solar_radiation_xf': {'floatList': {'value': [0.0]}},
                           'temperature_xf': {'floatList': {'value': [0.22027971]}},
                           'visibility_xf': {'floatList': {'value': [1.0]}},
                           'wind_speed_xf':